# Rewriting the code more compactly

This code does the same thing as "ols_simple" but it is written in a more object oriented way. While at this stage you may not be convinced why this is useful, it will become clearer as we turn the code into a full blown Monte Carlo simulation.

In [1]:
# loading modules
using LinearAlgebra
using Statistics
using Random

In [2]:
# defining composite types that
# serve as containers to store results

struct Parameters
    n::Int64
    beta_0::Float64
    beta_1::Float64
end

struct Sample
    x::Array{Float64,2}
    y::Array{Float64,1}
end

struct OLS_results
    bols::Array{Float64,1}
    se_hom::Array{Float64,1}
    se_het::Array{Float64,1}
    ci_hom::Array{Float64,1}
    ci_het::Array{Float64,1}
end

# Defining useful functions

In [3]:
"""
    sample(parms)

Creates one random sample according to linear model
Y = \beta_0 + \beta_1 X + e

Requires sample size and coefficients from Parameters object.

"""
function sample(parms::Parameters)

    # creating sample
    Random.seed!(42); # fixing random numbers for cross-code comparison

    # exogenous variables: simply set as normal rvs
    X = [ones(parms.n, 1) randn(parms.n)]
    e = randn(parms.n) 

    # creating endogenous variables    
    Y = X*[parms.beta_0; parms.beta_1] + e;
    
    # returning Sample object
    Sample(X, Y)
    
end

sample

In [4]:
"""
    ols(X, Y)

Computes OLS estimate, its standard error, and the corresponding 95% confidence interval.

Requires Nx2 dimensional data matrix X, and Nx1 dimensional data vector Y as input.
"""
function ols(X::Array{Float64,2}, Y::Array{Float64,1})
    
    xxinv = inv(X'*X)
   
    bols = xxinv*X'*Y
    
    ehat = Y - X*bols;
    shat = ehat'*ehat/length(ehat);

    Omegahat_hom = shat[1]*inv(X'*X);
    Omegahat_het = xxinv * (X'*Diagonal(ehat.^2)*X) * xxinv;

    se_hom = diag(Omegahat_hom).^(0.5)
    se_het = diag(Omegahat_het).^(0.5)
    
    ci_hom = [bols[2] - 1.96*se_hom[2], bols[2] + 1.96*se_hom[2]]
    ci_het = [bols[2] - 1.96*se_het[2], bols[2] + 1.96*se_het[2]]
    
    # returning OLS_results object
    OLS_results(bols, se_hom, se_het, ci_hom, ci_het)

end

ols (generic function with 1 method)

# OLS estimation

In [5]:
# setting parameters
parms = Parameters(1000, 24, 8);

In [6]:
# generate one sample
sample_data = sample(parms);

In [7]:
# ols estimation
myresults = ols(sample_data.x, sample_data.y);

In [8]:
println(myresults.bols)

[24.015845699885638, 8.047945398812304]


In [9]:
println(myresults.ci_hom)

[7.989025264173771, 8.106865533450838]
